# GeoPandas and OSM Integration

Integrating Google Earth Engine with local vector data and OpenStreetMap (OSM) allows for powerful hybrid workflows. This notebook demonstrates how to use `geopandas` for local vector manipulation and `osmnx` for retrieving OSM data to interact with Earth Engine.

In [ ]:
import ee
import geopandas as gpd
import osmnx as ox
import xarray as xr
import xee
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

## Retrieving OSM Data

We can use `osmnx` to download geometries (like city boundaries or parks) directly from OpenStreetMap.

In [ ]:
# Download a boundary from OSM
city_name = "Bangalore, India"
boundary = ox.geocode_to_gdf(city_name)

boundary.plot()
plt.title(f"Boundary of {city_name}")
plt.show()

## Using GeoPandas with Earth Engine

To use a GeoPandas geometry in Earth Engine, we convert it to an `ee.Geometry`.

In [ ]:
# Convert GeoPandas geometry to EE geometry
geom_coords = list(boundary.geometry.iloc[0].exterior.coords)
ee_geom = ee.Geometry.Polygon(geom_coords)

# Filter a GEE collection using this geometry
s2_collection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(ee_geom) \
    .filterDate('2023-01-01', '2023-02-01') \
    .select(['B4', 'B3', 'B2'])

print(f"Found {s2_collection.size().getInfo()} images.")

## Loading as Xarray with Spatial Clipping

Using `xee`, we can load the data directly cropped to our OSM-derived boundary.

In [ ]:
ds = xr.open_dataset(
    s2_collection.median(),
    engine='ee',
    geometry=ee_geom,
    scale=100
)

ds[['B4', 'B3', 'B2']].to_array().plot.imshow(robust=True)
plt.title("Sentinel-2 Median Composite over OSM Boundary")
plt.show()